In [83]:
import gzip
import csv
import pandas as pd


lehd_df = pd.read_csv('data\lodes\ca_wac_S000_JT00_2017.csv.gz',compression='gzip',header=0,sep=',',quotechar='"',error_bad_lines=False)


In [84]:
lehd_df

,w_geocode,C000,CA01,CA02,CA03,CE01,CE02,CE03,CNS01,CNS02,...,CFA02,CFA03,CFA04,CFA05,CFS01,CFS02,CFS03,CFS04,CFS05,createdate
0,60014001001007,49,23,14,12,19,9,21,0,0,...,0,0,0,0,0,0,0,0,0,20190825
1,60014001001008,3,0,1,2,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,20190825
2,60014001001017,6,0,3,3,0,0,6,0,0,...,0,0,0,0,0,0,0,0,0,20190825
3,60014001001024,16,1,10,5,3,2,11,0,0,...,0,0,0,0,0,0,0,0,0,20190825
4,60014001001026,3,0,1,2,0,2,1,0,0,...,0,0,0,0,0,0,0,0,0,20190825
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252322,61150411004035,10,0,4,6,6,3,1,0,0,...,0,0,0,0,0,0,0,0,0,20190825
252323,61150411004037,25,2,11,12,9,11,5,0,0,...,0,0,0,0,0,0,0,0,0,20190825
252324,61150411004047,17,7,9,1,8,6,3,0,0,...,0,0,0,0,0,0,0,0,0,20190825
252325,61150411004051,2,0,0,2,0,2,0,0,0,...,0,0,0,0,0,0,0,0,0,20190825


In [85]:
#convert w_geocode to string to make it easier to check
lehd_df['w_geocode'] = lehd_df['w_geocode'].astype(str)

In [86]:
# check if beginning of string matches SD county fips code
SDlehd_df = lehd_df[lehd_df['w_geocode'].str.match('6073')]

In [87]:
SDlehd_df

,w_geocode,C000,CA01,CA02,CA03,CE01,CE02,CE03,CNS01,CNS02,...,CFA02,CFA03,CFA04,CFA05,CFS01,CFS02,CFS03,CFS04,CFS05,createdate
174510,60730001001000,9,1,3,5,1,0,8,0,0,...,0,0,0,0,0,0,0,0,0,20190825
174511,60730001001003,1,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,20190825
174512,60730001001004,1,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,20190825
174513,60730001001010,4,0,0,4,0,2,2,0,0,...,0,0,0,0,0,0,0,0,0,20190825
174514,60730001001011,2,0,1,1,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,20190825
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192210,60730221003012,8,0,5,3,3,2,3,0,0,...,0,0,0,0,0,0,0,0,0,20190825
192211,60730221003013,1,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,20190825
192212,60730221003014,1,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,20190825
192213,60730221003015,44,2,31,11,9,11,24,0,0,...,0,0,0,0,0,0,0,0,0,20190825
